<a href="https://colab.research.google.com/github/SahithiK792/ML/blob/main/Lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded= files.upload()

for filename in uploaded.keys():
  print(filename)

Saving iotsim-air-quality-1 (1).csv to iotsim-air-quality-1 (1).csv
iotsim-air-quality-1 (1).csv


In [ ]:
!pip install lime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=1554af9de9b34f1004e1ccef59a5a33c930375236bd6323779305be37973883d
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Faster models that handle missing values
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import (
    HistGradientBoostingRegressor, HistGradientBoostingClassifier,
    StackingRegressor, StackingClassifier
)
from sklearn.linear_model import LinearRegression, LogisticRegression
import lime
import lime.lime_tabular


# Step 1: Load dataset and clean
data = pd.read_csv("iotsim-air-quality-1 (1).csv")

# Drop timestamp-like columns
for col in data.columns:
    if pd.api.types.is_string_dtype(data[col]) and "GMT" in str(data[col].iloc[0]):
        data = data.drop(columns=[col])

# Drop fully empty columns
data = data.dropna(axis=1, how='all')

print("🔎 Missing values per column before imputation:")
print(data.isnull().sum())

# Assume last column is target
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Identify categorical & numeric columns
categorical_cols = [c for c in X.columns if X[c].dtype == 'object']
numeric_cols = [c for c in X.columns if X[c].dtype != 'object']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)



# A1. Build lightweight stacking model

def build_stacking_model(is_classification=True):
    if is_classification:
        base_learners = [
            ('hgb', HistGradientBoostingClassifier(random_state=42))
        ]
        meta_model = LogisticRegression(max_iter=500)
        model = StackingClassifier(
            estimators=base_learners,
            final_estimator=meta_model,
            passthrough=False,
            n_jobs=-1
        )
    else:
        base_learners = [
            ('hgb', HistGradientBoostingRegressor(random_state=42))
        ]
        meta_model = LinearRegression()
        model = StackingRegressor(
            estimators=base_learners,
            final_estimator=meta_model,
            passthrough=False,
            n_jobs=-1
        )
    return model


# A2. Pipeline with preprocessing + stacking

def build_pipeline(is_classification=True):
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_cols),
            ('cat', categorical_transformer, categorical_cols)
        ]
    )

    stacking_model = build_stacking_model(is_classification)
    pipe = Pipeline([
        ('preprocess', preprocessor),
        ('stacking', stacking_model)
    ])
    return pipe



# A3. LIME Explanation

def explain_with_lime(pipeline, X_train, X_test, is_classification=True):
    if not LIME_AVAILABLE:
        print("⚠️ LIME not installed. Skipping explanation.")
        return None

    explainer = lime.lime_tabular.LimeTabularExplainer(
        training_data=np.array(X_train),
        feature_names=X_train.columns,
        class_names=np.unique(y_train) if is_classification else None,
        mode="classification" if is_classification else "regression"
    )

    i = 0
    exp = explainer.explain_instance(
        data_row=X_test.iloc[i].values,
        predict_fn=pipeline.predict_proba if is_classification else pipeline.predict
    )
    exp.show_in_notebook(show_table=True)
    return exp


# Main Program

if __name__ == "__main__":
    # Detect task type (classification if y is categorical)
    is_classification = y.dtype == 'object' or y.nunique() < 20

    # Build and fit pipeline
    pipe = build_pipeline(is_classification)
    pipe.fit(X_train, y_train)

    # Evaluate
    score = pipe.score(X_test, y_test)
    task_type = "Classification" if is_classification else "Regression"
    print(f"\n✅ {task_type} Pipeline Score:", score)

    # LIME explanation
    explain_with_lime(pipe, X_train, X_test, is_classification)


/usr/local/lib/python3.12/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:19: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


🔎 Missing values per column before imputation:
frame.len                          0
frame.protocols                    0
eth.src                            0
eth.dst                            0
ip.dst                             0
ip.src                             0
ip.flags                           0
ip.ttl                             0
ip.proto                           0
ip.checksum                        0
tcp.srcport                     6597
tcp.dstport                     6597
tcp.flags                       6597
tcp.window_size_value           6597
tcp.window_size_scalefactor     8733
tcp.checksum                    6597
tcp.options                     7184
tcp.pdu.size                   54840
udp.srcport                    49519
udp.dstport                    49519
label                              0
dtype: int64
